In [1]:
%matplotlib inline
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# orders contain order to user connection
fname_orders = "data/orders.csv"
fname_orders_products = "data/order_products__prior.csv"

In [ ]:
df_orders = pd.read_csv(fname_orders)

In [ ]:
df_orders.head()

I only care about the order to user mapping. Also, making sure the only eval_set contained is prior.

In [ ]:
df_orders = df_orders[['order_id', 'user_id']][df_orders.eval_set == 'prior']

Loading and cleaning the orders_products data set.

In [ ]:
df_orders_products = pd.read_csv(fname_orders_products)

In [ ]:
df_orders_products.head()

Only care about the order_id and product_id.

In [ ]:
df_orders_products = df_orders_products[['order_id', 'product_id']]

Mapping users to products based on order id.

In [ ]:
df_merged = df_orders_products.merge(df_orders, on='order_id')

In [ ]:
df_merged.head()

There's ~49k products, let's see if we can trim that number by looking at the cumulative orders for each product.

In [ ]:
total_product_count = df_orders_products.product_id.value_counts()
total_product_count

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,6))

total_product_count.plot(kind='hist', bins=50, ax=axes[0][0])
total_product_count[total_product_count < 10000].plot(kind='hist', bins=50, ax=axes[0][1])
total_product_count[total_product_count < 1000].plot(kind='hist', bins=50, ax=axes[1][0])
total_product_count[total_product_count < 100].plot(kind='hist', bins=50, ax=axes[1][1])

In [ ]:
total_product_count[total_product_count < 1000].count()

In [ ]:
total_product_count[total_product_count < 1000].sum() / total_product_count.sum()

If we cut off products with less than 1000 total purchases, we'd be losing 44k total products but 18% of total purchases.

In [ ]:
print(total_product_count[total_product_count < 500].count())
print(total_product_count[total_product_count < 500].sum() / total_product_count.sum())

If we cut off products with less than 500 total purchases, we'd be losing 41k total products but down to 11% of total purchases.

In [ ]:
included_product_ids = sorted(list(total_product_count.index[total_product_count > 500]))

In [ ]:
df_merged = df_merged[df_merged.product_id.isin(included_product_ids)]

In [ ]:
agg = df_merged[['user_id', 'product_id']].groupby(by=['user_id', 'product_id']).size()

In [ ]:
agg.head()

In [ ]:
len(agg)

In [ ]:
agg_unstack = agg.unstack()

In [ ]:
agg_unstack

In [3]:
%%time

# reading in orders dataset & slicing on relevant columns
df_orders = pd.read_csv(fname_orders)
df_orders = df_orders[['order_id', 'user_id']][df_orders.eval_set == 'prior']

# reading in product order dataset & slicing on relevant columns
df_orders_products = pd.read_csv(fname_orders_products)
df_orders_products = df_orders_products[['order_id', 'product_id']]

# merging above datasets on product id
df_merged = df_orders_products.merge(df_orders, on='order_id')

# counting how many times each product was ordered
total_product_count = df_orders_products.product_id.value_counts()

# filtering for products had more than 500 orders
included_product_ids = sorted(list(total_product_count.index[total_product_count > 500]))

# slicing the merged dataframe on most ordered products
df_merged = df_merged[df_merged.product_id.isin(included_product_ids)]

# counting how many times each user ordered each product
agg = df_merged[['user_id', 'product_id']].groupby(by=['user_id', 'product_id']).size()

# unstacking to create matrix
agg = agg.unstack()

CPU times: user 34.2 s, sys: 9.06 s, total: 43.3 s
Wall time: 43.3 s


In [ ]:
print("df_orders shape: {0}".format(df_orders.shape))
print("df_orders_products shape: {0}".format(df_orders_products.shape))
print("df_merged shape: {0}".format(df_merged.shape))
print("agg shape: {0}".format(agg.shape))
print("agg_unstack shape: {0}".format(agg_unstack.shape))

In [4]:
#agg_unstack.to_csv("/mnt/ssd/instacart_agg_unstack.csv")

In [5]:
agg.head()

product_id,1,10,23,25,28,32,34,37,45,49,...,49622,49628,49630,49640,49644,49652,49655,49667,49680,49683
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
agg.values

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,   1., ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,   1.],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [5]:
from sklearn.cluster import KMeans

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, test = train_test_split(agg.values, test_size=0.35)

MemoryError: 

In [9]:
model = KMeans(n_clusters=5, n_jobs=-1)

In [10]:
model.fit(agg.values)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').